### Preprocess the .txt files

In [6]:
import re
from tqdm.auto import tqdm

In [1]:
import pandas as pd

/home/adeptschneiderthedev/.miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [10]:
# Read the data from the TXT file
with open('TeleQnA_training.txt', 'r') as file:
    train_data = file.read()

In [11]:
# Convert the data to a pandas DataFrame
train_df = pd.read_json(train_data, orient='index')

/tmp/ipykernel_203835/2878740358.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  train_df = pd.read_json(train_data, orient='index')


In [12]:
print(train_df.index)

Index(['question 0', 'question 4', 'question 11', 'question 12', 'question 14',
       'question 19', 'question 23', 'question 29', 'question 37',
       'question 41',
       ...
       'question 8086', 'question 8096', 'question 8101', 'question 8108',
       'question 8110', 'question 8111', 'question 8114', 'question 8117',
       'question 8123', 'question 8136'],
      dtype='object', length=1461)


In [ ]:
train_df.to_csv('TeleQnA_training.csv', index=False)

In [2]:
# Read the data from the TXT file
with open('TeleQnA_testing1.txt', 'r') as file:
    data = file.read()

In [4]:
# Convert the data to a pandas DataFrame
test_df = pd.read_json(data, orient='index')

/tmp/ipykernel_203835/4079608287.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  test_df = pd.read_json(data, orient='index')


In [5]:
print(test_df.index)

Index(['question 8138', 'question 8153', 'question 8156', 'question 8160',
       'question 8165', 'question 8168', 'question 8186', 'question 8200',
       'question 8202', 'question 8210',
       ...
       'question 9944', 'question 9945', 'question 9949', 'question 9957',
       'question 9962', 'question 9974', 'question 9984', 'question 9986',
       'question 9992', 'question 9999'],
      dtype='object', length=366)


In [7]:
def extract_question_id_from_test_df_index(index):
    pattern = r'question (\d+)'
    match = re.search(pattern, index)
    if match:
        question_id = match.group(1)
    else:
        question_id = None
    return question_id


In [8]:
for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc='Processing rows'):
    question_id = extract_question_id_from_test_df_index(index)
    print(question_id)

Processing rows:   0%|          | 0/366 [00:00<?, ?it/s]

8138
8153
8156
8160
8165
8168
8186
8200
8202
8210
8211
8213
8216
8223
8230
8231
8243
8261
8263
8266
8269
8272
8285
8289
8293
8301
8307
8310
8311
8314
8318
8319
8335
8342
8347
8350
8354
8359
8360
8361
8362
8363
8364
8369
8371
8381
8389
8396
8401
8403
8405
8411
8417
8426
8429
8434
8437
8445
8455
8456
8460
8462
8476
8485
8493
8499
8503
8508
8523
8525
8531
8538
8541
8546
8552
8568
8570
8576
8578
8579
8598
8600
8601
8611
8614
8617
8620
8627
8635
8636
8638
8639
8641
8650
8654
8655
8657
8662
8663
8665
8672
8677
8683
8690
8693
8694
8696
8703
8716
8721
8723
8735
8742
8755
8762
8768
8770
8774
8779
8795
8798
8801
8806
8808
8814
8833
8854
8860
8862
8863
8868
8874
8875
8877
8882
8885
8886
8901
8912
8915
8942
8946
8947
8948
8950
8955
8957
8961
8962
8964
8965
8971
8972
8981
8987
9001
9010
9013
9014
9022
9023
9025
9028
9030
9043
9046
9048
9050
9052
9061
9073
9074
9077
9091
9095
9099
9101
9102
9112
9118
9124
9126
9129
9139
9145
9146
9167
9168
9174
9180
9187
9191
9204
9208
9213
9214
9219
9224
9225
9231


In [9]:
test_df.to_csv('TeleQnA_testing1_.csv', index=False)

## Install the required dependencies

In [1]:
!pip install -q torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.38.2 trl==0.4.7 accelerate einops tqdm scipy


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os
from dataclasses import dataclass, field
from typing import Optional

In [3]:
import torch
from datasets import load_dataset, load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)

In [4]:
from tqdm.notebook import tqdm

In [5]:
from trl import SFTTrainer

In [7]:
from huggingface_hub import interpreter_login

In [8]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  Y


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
dataset = load_dataset('csv', data_files='TeleQnA_training.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
# Iterate through the dataset and print the first 5 rows
for i in range(5):
    print(dataset['train'][i])

{'question': 'What is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]', 'option 1': 'To configure the MFAF to map data or analytics received by the MFAF to out-bound notification endpoints', 'option 2': 'To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints', 'option 3': 'To supply data or analytics from the MFAF to notification endpoints', 'option 4': 'To fetch data or analytics from the MFAF based on fetch instructions', 'answer': 'option 2: To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints', 'explanation': 'The Nmfaf_3daDataManagement_Deconfigure service operation is used to stop mapping data or analytics received by the MFAF to one or more out-bound notification endpoints.', 'category': 'Standards specifications', 'option 5': None}
{'question': 'How does a supporting UE attach to the same core network operator from whi

In [11]:
alpaca_prompt = """Below is an instruction that describes a task.
Use the options to provide an answer to the question.

### Question:
{}

### Option 1:
{}

### Option 2:
{}

### Option 3:
{}

### Option 4:
{}

### Option 5:
{}

### Answer:
{}

### Explanation:
{}

### Category:
{}
"""


def formatting_prompts_func(examples):
    questions = examples["question"]
    options1 = examples["option 1"]
    options2 = examples["option 2"]
    options3 = examples["option 3"]
    options4 = examples["option 4"]
    options5 = examples["option 5"]
    answers = examples["answer"]
    explanations = examples["explanation"]
    categories = examples["category"]
    texts = []
    for question, option1, option2, option3, option4, option5, answer, explanation, category in zip(questions, options1, options2, options3, options4, options5, answers, explanations, categories):
        text = alpaca_prompt.format(question, option1, option2, option3, option4, option5, answer, explanation, category)
        texts.append(text)
    return { "text" : texts}

In [12]:
dataset_ = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1461 [00:00<?, ? examples/s]

In [13]:
for i in range(5):
    print(dataset_['train'][i]['text'])

Below is an instruction that describes a task.
Use the options to provide an answer to the question.

### Question:
What is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]

### Option 1:
To configure the MFAF to map data or analytics received by the MFAF to out-bound notification endpoints

### Option 2:
To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints

### Option 3:
To supply data or analytics from the MFAF to notification endpoints

### Option 4:
To fetch data or analytics from the MFAF based on fetch instructions

### Option 5:
None

### Answer:
option 2: To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints

### Explanation:
The Nmfaf_3daDataManagement_Deconfigure service operation is used to stop mapping data or analytics received by the MFAF to one or more out-bound notification endpoints.

### Category:
Standards s

## Fine Tuning Code

In [14]:
base_model = "microsoft/phi-2"
new_model = "phi-2-telecom"

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = False
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = bnb_config,
    trust_remote_code = True,
    flash_attn = True,
    flash_rotary = True,
    fused_dense = True,
    low_cpu_mem_usage = True,
    # device_map = {"", 0},
    revision = "refs/pr/23"
)

model.config.use_cache = False
model.config.pretraining_tp = 1

model = prepare_model_for_kbit_training(model)

training_arguments = TrainingArguments(
    output_dir = "./phi-2-finetuned-model",
    num_train_epochs = 5,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 32,
    evaluation_strategy = "steps",
    eval_steps = 1500,
    logging_steps = 15,
    optim = "paged_adamw_8bit",
    learning_rate = 2e-4,
    lr_scheduler_type = "cosine",
    save_steps = 1500,
    warmup_ratio = 0.05,
    weight_decay = 0.01,
    max_steps = -1
)

peft_config = LoraConfig(
    r = 32,
    lora_alpha = 64,
    lora_dropout = 0.05,
    task_type = "CAUSAL_LM",
    target_modules = ["Wqkv", "fc1", "fc2"]
)


trainer = SFTTrainer(
    model = model,
    train_dataset = dataset_["train"],
    peft_config = peft_config,
    dataset_text_field = "text",
    tokenizer = tokenizer,
    args = training_arguments
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the n

Map:   0%|          | 0/1461 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
trainer.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.


Step,Training Loss,Validation Loss


TrainOutput(global_step=110, training_loss=0.9522436488758433, metrics={'train_runtime': 1241.4449, 'train_samples_per_second': 5.884, 'train_steps_per_second': 0.089, 'total_flos': 2.3515873323264e+16, 'train_loss': 0.9522436488758433, 'epoch': 4.82})

In [32]:
test_dataset = load_dataset('csv', data_files='TeleQnA_testing1.csv')

In [33]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'option 1', 'option 2', 'option 3', 'option 4', 'category', 'option 5'],
        num_rows: 366
    })
})

In [35]:
print(test_dataset['train'][1])

{'question': 'What does OTA REFSENS requirement ensure? [3GPP Release 18]', 'option 1': 'The accuracy of the receiver in filtering out out-of-band signals.', 'option 2': 'The ability of the receiver to receive a wanted signal in the presence of an interfering signal.', 'option 3': "The receiver's ability to receive an unwanted signal in the presence of a wanted signal.", 'option 4': 'The accuracy of the receiver in filtering out adjacent channel signals.', 'category': 'Standards specifications', 'option 5': 'The minimum mean power received at the RIB for a specified reference measurement channel.'}


In [36]:
test_dataset_alpaca_prompt = """Below is an instruction that describes a task.
Use the options to provide an answer to the question.

### Question:
{}

### Option 1:
{}

### Option 2:
{}

### Option 3:
{}

### Option 4:
{}

### Option 5:
{}
"""
def formatting_prompts_func_test_dataset(examples):
    questions = examples["question"]
    options1 = examples["option 1"]
    options2 = examples["option 2"]
    options3 = examples["option 3"]
    options4 = examples["option 4"]
    options5 = examples["option 5"]
    texts = []
    for question, option1, option2, option3, option4, option5 in zip(questions, options1, options2, options3, options4, options5):
        text = test_dataset_alpaca_prompt.format(question, option1, option2, option3, option4, option5)
        texts.append(text)
    return { "text" : texts}

In [37]:
test_dataset_ = test_dataset.map(formatting_prompts_func_test_dataset, batched = True,)

In [38]:
test_dataset_

DatasetDict({
    train: Dataset({
        features: ['question', 'option 1', 'option 2', 'option 3', 'option 4', 'category', 'option 5', 'text'],
        num_rows: 366
    })
})

In [39]:
print(test_dataset_['train'][0]['text'])

Below is an instruction that describes a task.
Use the options to provide an answer to the question.

### Question:
When can a gNB transmit a DL transmission(s) on a channel after initiating a channel occupancy? [3GPP Release 17]

### Option 1:
Regardless of the duration of the gap between the DL transmission(s) and any previous transmission(s) corresponding to the channel occupancy initiated by the gNB.

### Option 2:
If the gap between the DL transmission(s) and any previous transmission(s) corresponding to the channel occupancy initiated by the gNB is more than a threshold.

### Option 3:
Both option 1 and option 2

### Option 4:
None of the above

### Option 5:
None



In [40]:
from transformers import pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, device_map='auto', batch_size=8)

In [42]:
result = pipe(test_dataset_['train'][1]['text'])
print(result[0]['generated_text'])

Below is an instruction that describes a task.
Use the options to provide an answer to the question.

### Question:
What does OTA REFSENS requirement ensure? [3GPP Release 18]

### Option 1:
The accuracy of the receiver in filtering out out-of-band signals.

### Option 2:
The ability of the receiver to receive a wanted signal in the presence of an interfering signal.

### Option 3:
The receiver's ability to receive an unwanted signal in the presence of a wanted signal.

### Option 4:
The accuracy of the receiver in filtering out adjacent channel signals.

### Option 5:
The minimum mean power received at the RIB for a specified reference measurement channel.

### Answer:
option 5: The minimum mean power received at the RIB for a specified reference measurement channel.

### Explanation:
The OTA REFSENS requirement ensures that the receiver


In [51]:
trainer.model.save_pretrained("Adeptschneider/phi2-telecom_lora_adapter_weights")

In [49]:
trainer.push_to_hub("Adeptschneider/phi2-telecom_lora_adapter_weights")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.41k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/147M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Adeptschneider/phi-2-finetuned-model/commit/fcfae0da641d9da125dcb503d52d6692559101a6', commit_message='Adeptschneider/phi2-telecom_lora_adapter_weights', commit_description='', oid='fcfae0da641d9da125dcb503d52d6692559101a6', pr_url=None, pr_revision=None, pr_num=None)

In [52]:
HUGGING_FACE_REPO_NAME="Adeptschneider/phi2-telecom_lora_adapter_weights"

In [53]:
# Get peft config.
import transformers
from peft import PeftConfig
config = PeftConfig.from_pretrained(HUGGING_FACE_REPO_NAME)

# Get base model
model = transformers.AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                                          torch_dtype=torch.float16,      # GPTQ quantization requires fp16
                                                          return_dict=True)

# Load the Lora model.
from peft import PeftModel
model = PeftModel.from_pretrained(model, HUGGING_FACE_REPO_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [54]:
# Merge model and Lora adapter.
merged_model = model.merge_and_unload()

In [55]:
HUGGING_FACE_MERGED_REPO_NAME = 'phi2-telecom-fine-tuned-model'
# Push to HF Hub.
merged_model.push_to_hub(HUGGING_FACE_MERGED_REPO_NAME)
tokenizer.push_to_hub(HUGGING_FACE_MERGED_REPO_NAME)

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Adeptschneider/phi2-telecom-fine-tuned-model/commit/a9e6533f9709b62cb47693bdb286be6dc35022a5', commit_message='Upload tokenizer', commit_description='', oid='a9e6533f9709b62cb47693bdb286be6dc35022a5', pr_url=None, pr_revision=None, pr_num=None)